# Query Expansion

0) Just some imports

In [232]:
import re
import math
import numpy as np
import common as cm

# 1) Simple search engine

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [233]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### DONE
    def computeIDFs(self, documents):
        docs_total = len(documents)
        docs_occur = 0
        self.idfs = []
        for term in self.terms:
            for doc in documents:
                if term in doc.tokens:
                    docs_occur += 1
            self.idfs.append(math.log(docs_total/docs_occur, 2))
    
    ### DONE
    def computeCorM(self, documents):
        #create and initalize matrixA
        matrixA = np.zeros((len(self.terms), len(documents)))
        for rid,term in enumerate(self.terms):
            for cid, doc in enumerate(documents):
                for token in doc.tokens:
                    if token == term:
                        matrixA[rid][cid] += 1
        
        #calculate vector length
        tVec = []
        for row in matrixA:
            tVec.append(np.linalg.norm(row))
        
        #normalize
        for row in range(len(matrixA)):
            for col in range(len(matrixA[0])):
                matrixA[row][col] = matrixA[row][col]/tVec[row]
        
        #transpose matrixA
        matrixAT = np.transpose(matrixA)
        
        self.corM = np.matmul(matrixA, matrixAT)
        for i in range(len(self.corM)):
            self.corM[i][i] = -1
        
        

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [234]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [235]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [236]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = {} # Bag Of Words (BOW - number of term occurences)
        self.tf = {} # TF representation
        self.tf_idf = {} # TF-IDF representation

    ### DONE
    def computeBOW_Representation(self):
        self.bow = {}
        for term in self.tokens:
            if term in self.bow.keys():
                self.bow[term] += 1
            else:
                self.bow[term] = 1
    
    ### DONE
    def computeTF_Representation(self):
        self.tf = {}
        max_occur = 1
        for term in self.tokens:
            if term in self.tf.keys():
                self.tf[term] += 1
                max_occur = max(max_occur, self.tf[term])
            else:
                self.tf[term] = 1
        for key, value in self.tf.items():
            self.tf[key] = value/max_occur
    
    ### DONE
    def computeTF_IDF_Representation(self):
        self.tf_idf = {}
        for term, tfi in self.tf.items():
            try:
                idfi = self.dictionary.idfs[self.dictionary.terms.index(term)]
            except:
                idfi = 0
            self.tf_idf[term] = tfi*idfi
        
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]


1.4) Compute IDFs here

In [237]:
### DONE
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print(res[:5])

[['acquired', 3.137503523749935], ['access', 3.289506617194985], ['academic', 3.652076696579693], ['about', 3.8744691179161412], ['aaai', 5.459431618637297]]
[['zdm', -4.083600201617941], ['young', -4.082632923646146], ['you', -4.0816649967122265], ['york', -4.077786781901299], ['yahoo', -4.076815597050831]]


1.5) Compute the document representations (for each document run computeRepresentations())

In [238]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print(documents[0].bow)
print(documents[5].tf)
print(documents[7].tf_idf)

{'david': 1, 'aha': 1, 'machine': 4, 'learning': 4, 'page': 1, 'resource': 2, 'suggestion': 1, 'welcome': 1, 'wizrule': 1, 'zdm': 1, 'scientific': 1, 'ltd': 1, 'conference': 1, 'announcement': 1, 'course': 1, 'data': 1, 'repository': 1, 'description': 1, 'comprehensive': 1, 'from': 1, 'application': 1, 'tutorials': 1}
{'dataset': 0.6666666666666666, 'for': 0.3333333333333333, 'machine': 0.6666666666666666, 'learning': 0.6666666666666666, 'knowledge': 1.0, 'discovery': 0.6666666666666666, 'data': 0.6666666666666666, 'mining': 0.6666666666666666, 'add': 0.3333333333333333, 'update': 0.3333333333333333, 'the': 0.3333333333333333, 'mlnetois': 0.3333333333333333, 'event': 0.3333333333333333, 'related': 0.3333333333333333, 'acquisition': 0.3333333333333333, 'etc': 0.3333333333333333}
{'and': 0.19264507794239583, 'machine': -3.2870827025011664, 'learning': -3.0893902898214542, 'fall': -0.7317348032968899, 'this': -1.3215782233936009, 'course': -0.5172652122619856, 'cover': -0.524663794300051,

1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [320]:
### DONE
def getSimilarity(v1i, v2i):
    v1 = list(v1i.values())
    v2 = list(v2i.values())
    
    # make the same length
    while len(v1) < len(v2):
        v1.append(0)
    while len(v2) < len(v1):
        v2.append(0)
    dot = np.dot(v1, v2)
    normv1 = np.linalg.norm(v1)
    normv2 = np.linalg.norm(v2)
    cos = dot / (normv1*normv2)
    return cos


1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [322]:
query = "machine learning"
#query = "academic research"
#query = "international conference"
#query = "international conference washington"

In [323]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.8795110578261387 | DOC ID = 63
AI / Machine Learning Resources
 AI / Machine Learning Resources. General Machine Learning. The Journal
 of Machine Learning. MLnet Machine Learning Archive at GMD. The ... 


RANK = 2 WITH SIMILARITY = 0.8740788455239175 | DOC ID = 34
Machine Learning
 Machine Learning. Related Sites. Machine Learning Resources courtesy
 of David Aha A Machine Learning Tutorial a good overview of the ... 


RANK = 3 WITH SIMILARITY = 0.8666466425586407 | DOC ID = 77
Machine Learning
 Machine Learning. Machine Learning Home Page (Editor) Machine Learning Home
 Page (Publisher) Machine Learning Online by Kluwer Academic Publishers: ... 


RANK = 4 WITH SIMILARITY = 0.8587418957884306 | DOC ID = 28
Machine Learning
 6.858/18.428: Machine Learning. Available Lecture Notes. ... Defining models for
 machine learning. Learning conjunctions in the mistake-bounded model. ... 


RANK = 5 WITH SIMILARITY = 0.8328803549278195 | DOC ID = 29
Machine Learni

# 2) Query expansion

## 2.1) Correlation matrix

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [242]:
### DONE
dictionary.computeCorM(documents)
print(dictionary.corM)

[[-1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.         -1.          0.         ...  0.18898224  0.
   0.        ]
 [ 0.          0.         -1.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.18898224  0.         ... -1.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.         -1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -1.        ]]


2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [243]:
query = "machine"
#query = "algorithm"
# query = "learning"
# query = "conference"
# query = "research"
# query = "concept"

def suggestKeywords(query, dictionary):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    queri = dictionary.terms.index(query)
    ### DONE
    ranking = []
    print("Suggestions")
    for i, corr in enumerate(dictionary.corM[queri]):
        ranking.append([dictionary.terms[i], corr])
    ranking.sort(key=lambda x: -x[1])
    for i in ranking[:10]:
        print(i[0], i[1])
    pass
        
suggestKeywords(query, dictionary)

Suggestions
learning 0.9700552531915929
the 0.6711549305747254
description 0.5852726298569305
and 0.5653332650783621
for 0.44240101088746486
page 0.4135841780912734
resource 0.4085035786329266
research 0.3812617661967056
group 0.3793557371640793
home 0.3516916400856809


# 2.2) Rocchio algorithm

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [327]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    for key, one_tf_idf in qd.tf_idf.items():
        f1 = alpha*one_tf_idf
        f2 = beta*(1/len(rel_docs)*sum([documents[rel_id].tf_idf[key] for rel_id in rel_docs]))
        f3 = gamma*(1/len(nrel_docs)*sum([documents[nrel_id].tf_idf[key] for nrel_id in nrel_docs]))
        qd.tf_idf[key] = f1 + f2 - f3
        
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

RANK = 1 WITH SIMILARITY = 0.8760674903105631 | DOC ID = 63
AI / Machine Learning Resources
 AI / Machine Learning Resources. General Machine Learning. The Journal
 of Machine Learning. MLnet Machine Learning Archive at GMD. The ... 


RANK = 2 WITH SIMILARITY = 0.8706565468595463 | DOC ID = 34
Machine Learning
 Machine Learning. Related Sites. Machine Learning Resources courtesy
 of David Aha A Machine Learning Tutorial a good overview of the ... 


RANK = 3 WITH SIMILARITY = 0.8663006683370706 | DOC ID = 28
Machine Learning
 6.858/18.428: Machine Learning. Available Lecture Notes. ... Defining models for
 machine learning. Learning conjunctions in the mistake-bounded model. ... 


RANK = 4 WITH SIMILARITY = 0.863253443349555 | DOC ID = 77
Machine Learning
 Machine Learning. Machine Learning Home Page (Editor) Machine Learning Home
 Page (Publisher) Machine Learning Online by Kluwer Academic Publishers: ... 


RANK = 5 WITH SIMILARITY = 0.8402114903877352 | DOC ID = 29
Machine Learnin

# 2.3) WordNet

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [215]:
import nltk 
nltk.download()

from nltk.corpus import wordnet as wn

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [216]:
wn.synsets('machine')

[Synset('machine.n.01'),
 Synset('machine.n.02'),
 Synset('machine.n.03'),
 Synset('machine.n.04'),
 Synset('machine.n.05'),
 Synset('car.n.01'),
 Synset('machine.v.01'),
 Synset('machine.v.02')]

2.3.3) Display all definitions (.definition()) for synsets (machine)

In [224]:
for i in wn.synsets('machine'):
    print(i, i.definition())
#DONE

Synset('machine.n.01') any mechanical or electrical device that transmits or modifies energy to perform or assist in the performance of human tasks
Synset('machine.n.02') an efficient person
Synset('machine.n.03') an intricate organization that accomplishes its goals efficiently
Synset('machine.n.04') a device for overcoming resistance at one point by applying force at some other point
Synset('machine.n.05') a group that controls the activities of a political party
Synset('car.n.01') a motor vehicle with four wheels; usually propelled by an internal combustion engine
Synset('machine.v.01') turn, shape, mold, or otherwise finish by machinery
Synset('machine.v.02') make by machinery


2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [230]:
for i in wn.synsets('machine'):
    print(i, i.hypernyms())
#DONE

Synset('machine.n.01') [Synset('device.n.01')]
Synset('machine.n.02') [Synset('person.n.01')]
Synset('machine.n.03') [Synset('organization.n.01')]
Synset('machine.n.04') [Synset('mechanical_device.n.01')]
Synset('machine.n.05') [Synset('organization.n.01')]
Synset('car.n.01') [Synset('motor_vehicle.n.01')]
Synset('machine.v.01') [Synset('shape.v.02')]
Synset('machine.v.02') [Synset('produce.v.02')]


See: http://www.nltk.org/howto/wordnet.html
for more examples